## Clasificación de texto

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)
import torch

/Users/dragonnomada/.pyenv/versions/3.10.15/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.vocab_size

30522

In [41]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.vocab_size

50257

In [42]:
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=3)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
inputs = tokenizer("I am so sad, I am starving", return_tensors="pt")

inputs

{'input_ids': tensor([[   40,   716,   523,  6507,    11,   314,   716, 34253]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [55]:
outputs = model(**inputs)

outputs

SequenceClassifierOutputWithPast(loss=None, logits=tensor([[11.1155, -7.4522, -2.7242]], grad_fn=<IndexBackward0>), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None)

In [56]:
outputs.logits

tensor([[11.1155, -7.4522, -2.7242]], grad_fn=<IndexBackward0>)

In [57]:
with torch.no_grad():
    outputs = model(**inputs)

outputs

SequenceClassifierOutputWithPast(loss=None, logits=tensor([[11.1155, -7.4522, -2.7242]]), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None)

In [58]:
from torch.nn import functional

probs = functional.softmax(outputs.logits, dim=-1)

probs

tensor([[1.0000e+00, 8.6326e-09, 9.7609e-07]])

## Generación de texto

In [59]:
from transformers import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [61]:
inputs = tokenizer("Hi how are you today?", return_tensors="pt")

outputs = model(**inputs)

outputs

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -34.2417,  -34.3302,  -37.3032,  ...,  -43.0447,  -42.7168,
           -35.2204],
         [-111.0282, -111.7505, -117.2466,  ..., -119.6479, -120.1152,
          -114.1044],
         [ -89.3942,  -89.9615,  -92.8167,  ...,  -92.8855,  -96.1467,
           -90.6905],
         [-103.1975, -105.2266, -109.9585,  ..., -112.8601, -114.4420,
          -108.3585],
         [ -72.0004,  -73.5239,  -77.6296,  ...,  -84.3382,  -84.0815,
           -76.9029],
         [-109.2879, -108.5157, -110.3026,  ..., -119.1990, -118.4583,
          -106.9128]]], grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(layers=[DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer, DynamicLayer]), hidden_states=None, attentions=None, cross_attentions=None)

In [62]:
outputs.keys()

odict_keys(['logits', 'past_key_values'])

In [83]:
tokens_logits = outputs.logits
tokens_logits

tensor([[[ -34.2417,  -34.3302,  -37.3032,  ...,  -43.0447,  -42.7168,
           -35.2204],
         [-111.0282, -111.7505, -117.2466,  ..., -119.6479, -120.1152,
          -114.1044],
         [ -89.3942,  -89.9615,  -92.8167,  ...,  -92.8855,  -96.1467,
           -90.6905],
         [-103.1975, -105.2266, -109.9585,  ..., -112.8601, -114.4420,
          -108.3585],
         [ -72.0004,  -73.5239,  -77.6296,  ...,  -84.3382,  -84.0815,
           -76.9029],
         [-109.2879, -108.5157, -110.3026,  ..., -119.1990, -118.4583,
          -106.9128]]], grad_fn=<UnsafeViewBackward0>)

In [84]:
token_probs = torch.softmax(tokens_logits, dim=-1)
token_probs

tensor([[[4.0598e-03, 3.7158e-03, 1.9006e-04,  ..., 6.1012e-07,
          8.4688e-07, 1.5257e-03],
         [5.8671e-04, 2.8493e-04, 1.1689e-06,  ..., 1.0591e-07,
          6.6376e-08, 2.7066e-05],
         [1.0464e-05, 5.9334e-06, 3.4145e-07,  ..., 3.1872e-07,
          1.2222e-08, 2.8622e-06],
         [1.2808e-03, 1.6838e-04, 1.4833e-06,  ..., 8.1486e-08,
          1.6752e-08, 7.3469e-06],
         [8.2389e-03, 1.7955e-03, 2.9589e-05,  ..., 3.6109e-08,
          4.6674e-08, 6.1196e-05],
         [1.9439e-04, 4.2073e-04, 7.0464e-05,  ..., 9.6455e-09,
          2.0231e-08, 2.0900e-03]]], grad_fn=<SoftmaxBackward0>)

In [89]:
tokens_top = torch.topk(token_probs, 4)
tokens_top

torch.return_types.topk(
values=tensor([[[0.0761, 0.0642, 0.0355, 0.0332],
         [0.0827, 0.0694, 0.0525, 0.0509],
         [0.6753, 0.0811, 0.0452, 0.0224],
         [0.2560, 0.1301, 0.0952, 0.0424],
         [0.3864, 0.1496, 0.0824, 0.0345],
         [0.2775, 0.0697, 0.0260, 0.0241]]], grad_fn=<TopkBackward0>),
indices=tensor([[[  13,   11,  198,   12],
         [ 466,  881,  750,  460],
         [ 345,  356,  484,  262],
         [1804, 4203,   30, 1016],
         [  30, 1701,   11,   13],
         [ 198,  314, 1867, 1374]]]))

In [90]:
for indices, values in zip(tokens_top.indices, tokens_top.values):
    for ids in indices:
        print(tokenizer.decode(ids))

.,
-
 do much did can
 you we they the
 doing feeling? going
??",.

 I What How
